<a href="https://colab.research.google.com/github/dizzySummer/NLP-TensorFlow2.0/blob/master/twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install emoji

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [35]:
#!pip install emoji

import datetime
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import emoji
import os
import sys
#import fasttext
import re
import nltk 
nltk.download('punkt')
from nltk.corpus import stopwords

import itertools
import collections
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
       

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
import io

df_train = pd.read_csv('/content/drive/My Drive/train.csv')
df_test = pd.read_csv('/content/drive/My Drive/test.csv')
submission = pd.read_csv('/content/drive/My Drive/sample_submission.csv')
# Dataset is now stored in a Pandas Dataframe
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [0]:
train=df_train[['text','target']]
#null_data = train[train.isnull().any(axis=1)] there is no null value in test set after removing the id, keyword, location column
test=df_test[['id','text']]
pred=test

In [0]:
#import contractions list and remove it in the next step
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"thx"   : "thanks"
}

In [0]:
def remove_contractions(text):
    return contractions[text.lower()] if text.lower() in contractions.keys() else text

In [40]:
train['text']=train['text'].apply(remove_contractions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
#the original test dataset which does not have labels
pred=pred['text'].apply(remove_contractions)


In [0]:
# clean dataset
def clean_dataset(text):
    # To lowercase
    text = text.lower()
    # Remove hashtag while keeping hashtag text
    text = re.sub(r'#','', text)
    # Remove HTML special entities (e.g. &amp;)
    text = re.sub(r'\&\w*;', '', text)
    # Remove tickers
    text = re.sub(r'\$\w*', '', text)
    # Remove hyperlinks
    text = re.sub(r'https?:\/\/.*\/\w*', '', text)
    # Remove whitespace (including new line characters)
    text = re.sub(r'\s\s+','', text)
    text = re.sub(r'[ ]{2, }',' ',text)
    # Remove URL, RT, mention(@)
    text=  re.sub(r'http(\S)+', '',text)
    text=  re.sub(r'http ...', '',text)
    text=  re.sub(r'(RT|rt)[ ]*@[ ]*[\S]+','',text)
    text=  re.sub(r'RT[ ]?@','',text)
    text = re.sub(r'@[\S]+','',text)
    # Remove words with 2 or fewer letters
    text = re.sub(r'\b\w{1,2}\b', '', text)
    #&, < and >
    text = re.sub(r'&amp;?', 'and',text)
    text = re.sub(r'&lt;','<',text)
    text = re.sub(r'&gt;','>',text)
    # Insert space between words and punctuation marks
    text = re.sub(r'([\w\d]+)([^\w\d ]+)', '\1 \2',text)
    text = re.sub(r'([^\w\d ]+)([\w\d]+)', '\1 \2',text)
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    text= ''.join(c for c in text if c <= '\uFFFF') 
    text = text.strip()
    # Remove misspelling words
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    # Remove punctuation
    text = ' '.join(re.sub("[\.\,\!\?\:\;\-\=\/\|\'\(\']", " ", text).split())
    # Remove emoji
    text = emoji.demojize(text)
    text = text.replace(":"," ")
    text = ' '.join(text.split()) 
    text = re.sub("([^\x00-\x7F])+"," ",text)
    # Remove Mojibake (also extra spaces)
    text = ' '.join(re.sub("[^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a]", " ", text).split())
    return text

In [43]:
train['text'] =train['text'].apply(clean_dataset)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
pred=pred.apply(clean_dataset)

In [0]:
# I ended up with building my own stop word lists because NLTP simply removed all negation words which totally changed the meaning of the sentence. 
myOwnStopList=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'what','how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'do', 'should', "should", 'now', 'd', 'm', 'o', 're', 've', 'y', 'ain', "are", 'could', "was",
'would','have','get','got','getting','one','two','still','going']

In [46]:
#stop = stopwords.words('english')
#stop+=['get','got','getting','one','two','would','still','could','going']#customized stop word list
#stop = [e for e in stop if e not in ("n't", "not", "no")]

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.

train['text'] = train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (myOwnStopList)]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
pred=pred.apply(lambda x: ' '.join([word for word in x.split() if word not in (myOwnStopList)]))

In [48]:
!pip --no-cache-dir install tokenizers==0.4.2 #you get only a warning, just ignore it

In [0]:
#!pip install tokenizers #hugging face tokenizer
from tokenizers import (ByteLevelBPETokenizer,
                            CharBPETokenizer,
                            SentencePieceBPETokenizer,
                            BertWordPieceTokenizer)
tokenizer = BertWordPieceTokenizer()

path="/content/drive/My Drive/tobeTrained.txt"
#set vocab_size to 15000 as the len(train_set)was something like 12500 
tokenizer.train(files=path, vocab_size=15_000, min_frequency=2)

In [50]:
tokenizer.save(".", "/content/drive/My Drive/newBert")

['/content/drive/My Drive/newBert-vocab.txt']

In [51]:
#token.ids  PAD-->0, UNK-->1, CLS-->2 (start of sentence),SEP-->3 (end of sentence),MASK-->4
tokenizer = BertWordPieceTokenizer(
    "/content/drive/My Drive/newBert-vocab.txt",
    lowercase=True, 
    unk_token="<unk>",
    sep_token="</s>",
    cls_token="<s>",
)

tokenizer.add_special_tokens(["<pad>", "<mask>"])# same as default, unnecessary?

2

In [52]:
print(tokenizer)

Tokenizer(vocabulary_size=9938, model=BertWordPiece, add_special_tokens=True, unk_token=<unk>, sep_token=</s>, cls_token=<s>, clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=True, wordpieces_prefix=##)


In [0]:
#output = tokenizer.encode(press2[0])
#print(output.tokens)
#print(output.ids)

In [0]:
# As we don't have label for test dataset, I will split train dataset as trainsub and testsub dataset.
from sklearn.model_selection import train_test_split
X_subtrain, X_subtest, y_subtrain, y_subtest = train_test_split(
  train['text'],train['target'], test_size=0.2, random_state=42)

In [55]:
print(X_subtrain)

4996    courageous honest analysis need use atomic bom...
3263     wld shame golf cart became engulfed boycottbears
4907    tell rescind medals given soldiers massacre wo...
2855          worried drought might affect extreme dampen
4716                    lava blastpower red pantherattack
                              ...                        
5226           many obliteration servers always like play
5390    panic attack enough money drugs alcohol wanted...
860     omron automatic blood pressure monitor standar...
7603    officials say quarantine place alabama home po...
7270          moved england five years ago whirlwind time
Name: text, Length: 6090, dtype: object


In [0]:
def bert_token(texts,max_len=128): 
    all_input_ids=[]
    all_mask_ids=[]
    all_seg_ids=[]
    for token in texts: 
    
        input_ids=tokenizer.encode(token).ids
        mask_ids = [1] * len(input_ids)
        seg_ids = [0] * len(input_ids)
        padding = [0] * (max_len - len(input_ids))
        input_ids += padding
        mask_ids += padding
        seg_ids += padding
        all_input_ids.append(input_ids)
        all_mask_ids.append(mask_ids)
        all_seg_ids.append(seg_ids)

    
    return np.array(all_input_ids), np.array(all_mask_ids), np.array(all_seg_ids)
  

In [0]:
train_input=bert_token(X_subtrain,max_len=32)
test_input=bert_token(X_subtest,max_len=32)
pred=bert_token(pred,max_len=32)

In [0]:
def build_model(bert_layer, max_len=128):
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    mask_ids = Input(shape=(max_len,), dtype=tf.int32, name="mask_ids")
    seg_ids = Input(shape=(max_len,), dtype=tf.int32, name="seg_ids")

    pooled_output, sequence_output = bert_layer([input_ids,mask_ids,  seg_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_ids,  mask_ids,  seg_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [59]:
%%time

module_url = "https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 9.21 s, sys: 1.53 s, total: 10.7 s
Wall time: 10.6 s


In [60]:
model = build_model(bert_layer, max_len=32)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 32)]         0                                            
__________________________________________________________________________________________________
mask_ids (InputLayer)           [(None, 32)]         0                                            
__________________________________________________________________________________________________
seg_ids (InputLayer)            [(None, 32)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_ids[0][0]                  
                                                                 mask_ids[0][0]               

In [0]:
#Can I use TensorBoard with Google Colab?
#Tensorboard: ValueError: Duplicate plugins for name projector #22676
#https://github.com/pytorch/pytorch/issues/22676

logs_base_dir = "./logs"
logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

In [62]:
model.fit(
    train_input, 
    y_subtrain,
    callbacks=[tensorboard_callback],
    validation_data=(test_input, y_subtest),
    epochs=20,
    batch_size=10
)

Train on 6090 samples, validate on 1523 samples
Epoch 1/20
6090/6090 [==============================] - 128s 21ms/sample - loss: 0.6716 - accuracy: 0.5952 - val_loss: 0.6715 - val_accuracy: 0.5660
Epoch 2/20
6090/6090 [==============================] - 100s 16ms/sample - loss: 0.4823 - accuracy: 0.7816 - val_loss: 0.6123 - val_accuracy: 0.6888
Epoch 3/20
6090/6090 [==============================] - 100s 16ms/sample - loss: 0.4809 - accuracy: 0.7479 - val_loss: 0.7527 - val_accuracy: 0.5844
Epoch 4/20
6090/6090 [==============================] - 100s 16ms/sample - loss: 0.3021 - accuracy: 0.8764 - val_loss: 0.7497 - val_accuracy: 0.6573
Epoch 5/20
6090/6090 [==============================] - 100s 16ms/sample - loss: 0.1478 - accuracy: 0.9509 - val_loss: 0.7790 - val_accuracy: 0.6927
Epoch 6/20
6090/6090 [==============================] - 100s 16ms/sample - loss: 0.1257 - accuracy: 0.9586 - val_loss: 0.8813 - val_accuracy: 0.7065
Epoch 7/20
6090/6090 [==============================] - 10

In [0]:
model.save('BertModel.h5')
pred = model.predict(pred)
model.load_weights('BertModel.h5')

In [64]:
submission['target'] =pred.round().astype(int)
print(submission['target'])

0       0
1       1
2       0
3       0
4       1
       ..
3258    1
3259    0
3260    0
3261    0
3262    0
Name: target, Length: 3263, dtype: int64


In [66]:
!kill 936

/bin/bash: line 0: kill: (936) - No such process


In [67]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 936), started 1:29:46 ago. (Use '!kill 936' to kill it.)

<IPython.core.display.Javascript object>